In [9]:
from utils import load_cddd_encoder

cddd_enc = load_cddd_encoder('output_test/models/cddd_encoder.pt')
cddd_enc

CDDDEncoder(
  (encoder): Encoder(
    (gru1): GRU(42, 512, batch_first=True)
    (gru2): GRU(512, 1024, batch_first=True)
    (gru3): GRU(1024, 2048, batch_first=True)
    (dropout): Dropout(p=0.15, inplace=False)
    (latent_projection): Linear(in_features=3584, out_features=512, bias=True)
    (gaussian_noise): GaussianNoise()
  )
)

In [13]:
cddd_enc(['CO', 'CN']).shape

(2, 512)

In [3]:
import pandas as pd

df = pd.read_csv('data/700k_all_chembl.smi.zst', header=None, names=['smiles'])
print(df.info())
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 698595 entries, 0 to 698594
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   smiles  698595 non-null  object
dtypes: object(1)
memory usage: 5.3+ MB
None


,smiles
0,CC1(C)CCC(C)(C)c2cc(-c3cccc(-c4ccc(C(=O)O)cc4)...
1,Cc1cc2c(cc1C1=NOC(c3ccc(C(=O)O)cc3)C1)C(C)(C)C...
2,Cc1ccc(-c2ccc(C(=O)O)cc2)cc1-c1ccc2c(c1)C(C)(C...
3,Cc1cc2c(cc1-c1ncc(-c3ccc(C(=O)O)cc3)s1)C(C)(C)...
4,Cc1ccc(-c2ccc(C(=O)O)cc2)cc1-c1cc2c(cc1C)C(C)(...
5,Cc1cc2c(cc1-c1cccc(-c3ccc(C(=O)O)cc3)n1)C(C)(C...
6,CC1(C)CCC(C)(C)c2cc(-c3cccc(-c4ccc(C(=O)O)cc4)...
7,CC1(C)CCC(C)(C)c2cc(-c3ccc4cc(C(=O)O)ccc4c3)ccc21
8,CC(=Cc1ccc(C(=O)O)cc1)c1ccc2c(c1)C(C)(C)CCC2(C)C
9,CC(C)=C(c1ccc(C(=O)O)cc1)c1ccc2c(c1)C(C)(C)CCC...


In [4]:
df.sample(1000).to_csv('data/1k_test.smi', index=False, header=False)

In [ ]:
df.loc['cddd'] = None
df.loc[:9, 'cddd'] = results

In [ ]:
df.loc[:9, 'cddd'].apply(np.squeeze)

In [ ]:
df['cddd']